In [1]:
##Cells labeled "from github" were obtained from "https://github.com/EncinasBartos/TBarrier"


from matplotlib import pyplot as plt
import pandas as pd
import netCDF4
from scipy.io import savemat
import scipy.io as sio
import numpy as np
from tqdm.notebook import tqdm

In [3]:
## from github


def convert_meters_per_second_to_deg_per_day(X, Y, U_ms, V_ms):
    
    '''
    Converts units of velocity from m/s to deg/day. The units of the velocity field must 
    match the units of the grid coordinates and time.
    
    Parameters:
        X:       array(Ny, Nx), X-meshgrid.
        Y:       array(Ny, Nx), Y-meshgrid.
        U_ms:    array(Ny, Nx, Nt), x-component of velocity field in m/s
        V_ms:    array(Ny, Nx, Nt), y-component of velocity field in m/s
         
    Returns:
        U_degday:    array(Ny, Nx, Nt), x-component of velocity field in deg/day
        V_degday:    array(Ny, Nx, Nt), y-component of velocity field in deg/day
    '''
    
# import numpy

    import numpy as np
    # import math tools
    from math import cos, pi
    
    # Velocity field
    U_degday, V_degday = np.nan*U_ms.copy(), np.nan*V_ms.copy()
    
    # Radius of the earth
    earthRadius = 6371*(10**3)
    
    # Iterate over grid
    for i in tqdm(range(X.shape[0]),desc = 'converting units'):
        for j in range(Y.shape[1]):
            arg_cos = Y[i,j]*(pi/180)
            if Y[i,j] >85:
                U_degday[i,j,:] = 0
                V_degday[i,j,:] = 0
            else:                    
                U_degday[i, j, :] = (U_ms[i, j, :] / (cos(arg_cos)*(earthRadius)))*180*3600*24/pi
            V_degday[i, j, :] = (V_ms[i, j, :] / earthRadius)*180*3600*24/pi
            if U_ms[i,j,0] > 10**10:
                U_degday[i,j,:]= 0
                V_degday[i,j,:]= 0
        
    return U_degday, V_degday

In [4]:
##main
##download .NC file, extract data, convert units and read into "AVISO.mat"

##path in PC
path0 = "C:/Mario/TBarrier/TBarrier/Untitled Folder/"

##names of download files:
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_1676300217765.nc"  ## 21 days, 1 day interval 
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1676727285735.nc"    ##21 days, 6h intervals
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1679668384053.nc"    ##7 days, 6h intervals
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1679668580982.nc"   ##14 days, 6h intervals
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1679668816882.nc"  ##28 days, 6h intervals
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1683123882852.nc" ##21 days, 6 hours intervals, zoomed 1
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1683124027239.nc" ##21 days, 6 hours intervals, zoomed 2
#download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1683124174448.nc" ##21 days, 6 hours intervals, zoomed 3
download_file = "cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1683124226176.nc" ##21 days, 6 hours intervals, zoomed 4

##path of dowload file
file_location = '%sLAVD/%s' % (path0, download_file)
data_raw = netCDF4.Dataset(file_location)

##extract data
U_ms = np.array(data_raw['uo'])
V_ms = np.array(data_raw['vo'])
x = data_raw['longitude'][:]
y = data_raw['latitude'][:]
t = data_raw['time'][:]/24-data_raw['time'][0]/24
X, Y = np.meshgrid(x, y)

U_ms = np.swapaxes(U_ms,0,2)
U_ms = np.swapaxes(U_ms,1,3)
V_ms = np.swapaxes(V_ms,0,2)
V_ms = np.swapaxes(V_ms,1,3)

##convert units
u, v = convert_meters_per_second_to_deg_per_day(X, Y, U_ms, V_ms) 

##read data into "AVISO.mat"
data = {'u': u, 'v': v, 'x': X, 'y': Y, 't': t}
savemat("AVISO.mat", data)
mat = sio.loadmat('AVISO.mat')

print("done")

converting units:   0%|          | 0/121 [00:00<?, ?it/s]

done
